In [ ]:
!pip install hist
!pip install lmfit
!pip install xraylib

In [ ]:
!wget -O 2024_11_11_V20_I300_T100_CuNiZn.mca "https://raw.githubusercontent.com/simonemanti/incontri-fisica-2024/main/2024_11_14_V20_I300_T100_CuNiZn.mca"

In [ ]:
import glob

import json

import hist

from lmfit import Parameters
from lmfit.models import PolynomialModel, GaussianModel, ExponentialModel, PowerLawModel, RectangleModel

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

import numpy as np

import os

import pandas as pd

import re

from scipy.signal import find_peaks

from uncertainties import unumpy as unp

import xraylib


def load_mca_file(mcafile, rebin=1):

    df = pd.read_csv(mcafile, encoding='cp1252')
    
    counts = []
    save_data = False

    for row in df.to_numpy():
        if row[0] == '<<END>>':
            save_data = False
        if save_data:
            counts.append(float(row[0]))
        if row[0] == '<<DATA>>':
            save_data = True

    counts = np.array(counts, dtype=int)    
    bins = np.arange(8193)
    
    histogram = hist.Hist(
        hist.axis.Variable(bins, name="Ch", label="Channels"),
        storage=hist.storage.Weight(),
    )

    bin_centers = histogram.axes[0].centers

    for center, count in zip(bin_centers, counts):
        histogram.fill(np.full(count, center))

    return histogram[::hist.rebin(rebin)]

# Plot Spettro XRF in canali

In [ ]:
rebin = 4
histogram = load_mca_file('2024_11_14_V20_I300_T100_CuNiZn.mca', rebin=rebin)

fig, ax = plt.subplots(figsize=(7,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")
# ax.set_yscale('log')

centers = histogram.axes[0].centers
counts = histogram.values()

ax.step(centers, counts, where='mid', c='k', lw=1, label='Dati')

ax.legend(loc='upper right')
ax.set_xlim(2000,4000)
ax.set_ylim(0,counts.max()*1.2)
ax.set_xlabel('Canali')
ax.set_ylabel(f'Conteggi / {rebin} Canali')

# Fit Spettro XRF in Canali

Definizione del modello per la funzione di fit:

In [ ]:
model_1 = PolynomialModel(degree=2)

model_1 += GaussianModel(prefix='CuKa_')



Parametri del modello:

In [ ]:
params_1 = Parameters()

params_1.add('c0', value=1)
params_1.add('c1', value=1)
params_1.add('c2', value=1e-2)

params_1.add('CuKa_amplitude', value=1e4, min=0)
params_1.add('CuKa_center', value=2780)
params_1.add('CuKa_sigma', value=30, min=1, max=100)



Fit:

In [ ]:
ind = np.where(counts > 0)[0]
x = centers[ind]
y = counts[ind]

result_1 = model_1.fit(y, params_1, x=x, weights=1/np.sqrt(y))
print(result_1.fit_report(show_correl=False))

Plot risultato del fit:

In [ ]:
fig, ax = plt.subplots(figsize=(7,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")
# ax.set_yscale('log')

centers = histogram.axes[0].centers
counts = histogram.values()

x_i = np.linspace(x[0],x[-1], 3000)
fit_i = model_1.eval(result_1.params, x=x_i)

components = result_1.eval_components(x=x_i)

for key in components.keys():
    ax.plot(x_i, components[key], lw=0.6)

ax.plot(x_i, fit_i, c='r', lw=1, label='Fit')
ax.errorbar(centers, counts, np.sqrt(counts), fmt='o', c='k', lw=1., ms=1.5, capsize=0, label='Dati')

ax.legend(loc='upper right')
ax.set_xlim(2400,3500)
ax.set_ylim(0,counts.max()*1.2)
# ax.set_ylim(10,1.e4)
ax.set_xlabel('Canali')
ax.set_ylabel(f'Conteggi / {rebin} Canali')

# Fit funzione di Calibrazione

Centri dei picchi per le linee Ka:

In [ ]:
params_1 = result_1.params

canali = [param.value for param in params_1.values() if 'Ka_center' in param.name]
canali

Energie di riferimento:

In [ ]:
ref_energies = []

ref_energies.append(xraylib.LineEnergy(xraylib.SymbolToAtomicNumber('Ni'), xraylib.KA_LINE) * 1e3)
ref_energies.append(xraylib.LineEnergy(xraylib.SymbolToAtomicNumber('Cu'), xraylib.KA_LINE) * 1e3)
ref_energies.append(xraylib.LineEnergy(xraylib.SymbolToAtomicNumber('Zn'), xraylib.KA_LINE) * 1e3)

ref_energies

Plot centri vs energie:

In [ ]:
fig, ax = plt.subplots(figsize=(4,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")

ax.errorbar(canali, ref_energies, fmt='o', c='k', lw=1, ms=3)

ax.set_xlabel('Canali')
ax.set_ylabel('Energie [eV]')

Definizione del modello di fit per la funzione di calibrazione:

In [ ]:
model_2 = ???

Parametri della funzione di calibrazione

In [ ]:
???

Fit funzione di calibrazione:

In [ ]:
x = ?
y = ?

result_2 = model_2.fit(y, params_2, x=x)
print(result_2.fit_report(show_correl=False))

Plot funzione di Calibrazione:

In [ ]:
fig, ax = plt.subplots(figsize=(4,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")

ax.errorbar(canali, ref_energies, fmt='o', c='k', lw=1, ms=3)

x_i = np.linspace(2400,3200, 1000)
fit_i = model_2.eval(result_2.params, x=x_i)

ax.plot(x_i, fit_i, c='r')

ax.set_xlim(2500,3100)

ax.set_xlabel('Canali')
ax.set_ylabel('Energie [eV]')

# Calibrazione Spettro in energie

$$

Energie = offset + gain \cdot Canali

$$

In [ ]:
offset = result_2.params['c0'].value
gain = result_2.params['c1'].value

energies = ?

Plot Spettro XRF calibrato:

In [ ]:
fig, ax = plt.subplots(figsize=(7,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")
# ax.set_yscale('log')

ax.axvline(ref_energies[0], c='k', lw=1, ls='--')
ax.axvline(ref_energies[1], c='k', lw=1, ls='--')
ax.axvline(ref_energies[2], c='k', lw=1, ls='--')

ax.step(energies, counts, where='mid', c='k', lw=1)

# ax.set_xlim(0,8192)
ax.set_xlim(7e3,10e3)
ax.set_ylim(0,counts.max()*1.2)
ax.set_xlabel('Energie [keV]')
ax.set_ylabel(f'Conteggi / {rebin} Canali')


# Fit spettro calibrato

In [ ]:
model_3 = ???

In [ ]:
ref_energies = []

ref_energies.append(xraylib.LineEnergy(xraylib.SymbolToAtomicNumber('Ni'), xraylib.KA_LINE) * 1e3)
ref_energies.append(xraylib.LineEnergy(xraylib.SymbolToAtomicNumber('Ni'), xraylib.KB_LINE) * 1e3)
ref_energies.append(xraylib.LineEnergy(xraylib.SymbolToAtomicNumber('Cu'), xraylib.KA_LINE) * 1e3)
ref_energies.append(xraylib.LineEnergy(xraylib.SymbolToAtomicNumber('Cu'), xraylib.KB_LINE) * 1e3)
ref_energies.append(xraylib.LineEnergy(xraylib.SymbolToAtomicNumber('Zn'), xraylib.KA_LINE) * 1e3)
ref_energies.append(xraylib.LineEnergy(xraylib.SymbolToAtomicNumber('Zn'), xraylib.KB_LINE) * 1e3)

ref_energies = np.array(ref_energies)
ref_energies

In [ ]:
delta = 100

params_3 = Parameters()

params_3.add('ENC', value=120, min=50, max=200)
params_3.add('FF', value=0.1, min=0, max=0.2)
params_3.add('SiW', value=3.81, vary=False)

params_3.add('c0', value=1)
params_3.add('c1', value=1)
params_3.add('c2', value=1e-2)

params_3.add('CuKa_amplitude', value=1e4, min=0)
params_3.add('CuKa_center', value=ref_energies[2], min=ref_energies[2]-delta, max=ref_energies[2]+delta)
params_3.add('CuKa_sigma', value=30, expr=f'sqrt((ENC/2.35)**2 + FF * CuKa_center * SiW)')

In [ ]:
ind = np.where(counts > 0)[0]
x = energies[ind]
y = counts[ind]

result_3 = model_3.fit(y, params_3, x=x, weights=1/np.sqrt(y))
print(result_3.fit_report(show_correl=False))

In [ ]:
fig, ax = plt.subplots(figsize=(7,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")

centers = histogram.axes[0].centers
counts = histogram.values()

x_i = np.linspace(x[0],x[-1], 3000)
fit_i = model_3.eval(result_3.params, x=x_i)

components = result_3.eval_components(x=x_i)

ax.axvline(ref_energies[0], c='k', lw=1, ls='--')
ax.axvline(ref_energies[1], c='k', lw=1, ls='--')
ax.axvline(ref_energies[2], c='k', lw=1, ls='--')
ax.axvline(ref_energies[3], c='k', lw=1, ls='--')
ax.axvline(ref_energies[4], c='k', lw=1, ls='--')
ax.axvline(ref_energies[5], c='k', lw=1, ls='--')

for key in components.keys():
    ax.plot(x_i, components[key], lw=0.6)

ax.errorbar(energies, counts, np.sqrt(counts), fmt='o', c='k', lw=1., ms=1.5, capsize=0, label='Dati')
ax.plot(x_i, fit_i, c='r', lw=1, label='Fit')

ax.legend(loc='upper right')
ax.set_xlim(7000,10000)
ax.set_ylim(0,counts.max()*1.2)
# ax.set_yscale('log')
# ax.set_ylim(10,1.e4)
ax.set_xlabel('Energie [eV]')
ax.set_ylabel(f'Conteggi / {rebin} Canali')

# Calcolo Residui

In [ ]:
params_3 = result_3.params

energie_fit = np.array([param.value for param in params_3.values() if 'center' in param.name])
err_energie_fit = np.array([param.stderr for param in params_3.values() if 'center' in param.name])
labels = [param.name.split('_')[0] for param in params_3.values() if 'center' in param.name]

energie_fit

In [ ]:
fig, ax = plt.subplots(figsize=(7,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")

residui = energie_fit - ref_energies

ax.axhline(0, c='k', lw=1, ls='--')

ax.errorbar(range(6), residui, err_energie_fit, fmt='o', c='k', ms=3, capsize=2)

ax.set_xticks(range(6))
ax.set_xticklabels(labels)
ax.set_ylabel('Residui [eV]')


In [ ]:
REBIN = 4

amplitudes_values = [param.value / REBIN for param in params_3.values() if 'Ka_amplitude' in param.name]
amplitudes_errors = [param.stderr / REBIN for param in params_3.values() if 'Ka_amplitude' in param.name]

amplitudes = unp.uarray(amplitudes_values, amplitudes_errors)
wts = amplitudes / amplitudes.sum() * 100

In [ ]:
fig, ax = plt.subplots(figsize=(5,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")
ax.set_axisbelow(True)

ax.errorbar(range(3), unp.nominal_values(wts), unp.std_devs(wts), fmt='o', ms=4, label='XRF')
ax.errorbar(range(3), [12,64,24], fmt='o', ms=4, label='Nominal')

ax.legend()
ax.set_xlim(-0.5,2.5)
ax.set_ylim(0,100)
ax.set_xticks([0,1,2])
ax.set_xticklabels(['Ni','Cu','Zn'])
ax.set_ylabel('Concentrazione  %')

# Monete

In [ ]:
!wget -O 2024_11_11_V20_I300_T100_1euro.mca "https://raw.githubusercontent.com/simonemanti/incontri-fisica-2024/main/2024_11_14_V20_I300_T100_1euro.mca"
!wget -O 2024_11_11_V20_I300_T100_2euro.mca "https://raw.githubusercontent.com/simonemanti/incontri-fisica-2024/main/2024_11_14_V20_I300_T100_2euro.mca"

In [ ]:
rebin = 4
histogram_1 = load_mca_file('2024_11_14_V20_I300_T100_1euro.mca', rebin=rebin)
histogram_2 = load_mca_file('2024_11_14_V20_I300_T100_2euro.mca', rebin=rebin)

fig, ax = plt.subplots(figsize=(7,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")
# ax.set_yscale('log')

centers_1 = histogram_1.axes[0].centers
counts_1 = histogram_1.values()
energies_1 = gain * centers_1 + offset

centers_2 = histogram_2.axes[0].centers
counts_2 = histogram_2.values()
energies_2 = gain * centers_2 + offset

ax.step(energies_1, counts_1, where='mid', c='C0', lw=1.5, label='?')
ax.step(energies_2, counts_2, where='mid', c='C1', lw=1.5, label='?')

ax.legend(loc='upper right')
ax.set_xlim(7000,10000)
ax.set_ylim(0,counts_1.max()*1.2)
ax.set_xlabel('Energie [eV]')
ax.set_ylabel(f'Conteggi / {np.diff(energies_1)[0]:.0f} eV')

## Moneta 1

In [ ]:
ind = np.where(counts_1 > 0)[0]
x = energies_1[ind]
y = counts_1[ind]

result_4 = model_3.fit(y, params_3, x=x, weights=1/np.sqrt(y))
print(result_4.fit_report(show_correl=False))

In [ ]:
REBIN = 4
width = 1

params_4 = result_4.params

amplitudes_values_1 = [param.value / REBIN / width for param in params_4.values() if 'Ka_amplitude' in param.name]
amplitudes_errors_1 = [param.stderr / REBIN / width for param in params_4.values() if 'Ka_amplitude' in param.name]

amplitudes_1 = unp.uarray(amplitudes_values_1, amplitudes_errors_1)
wts_1 = amplitudes_1 / amplitudes_1.sum() * 100

fig, ax = plt.subplots(figsize=(5,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")
ax.set_axisbelow(True)

ax.errorbar(range(3), unp.nominal_values(wts_1), unp.std_devs(wts_1), fmt='o', ms=4, label='XRF')
ax.errorbar(range(3), [25,75,0], fmt='o', ms=4, label='Nominal')
# ax.errorbar(range(3), [5,75,20], fmt='o', ms=4, label='Nominal')

ax.legend()
ax.set_xlim(-0.5,2.5)
ax.set_ylim(0,100)
ax.set_xticks([0,1,2])
ax.set_xticklabels(['Ni','Cu','Zn'])
ax.set_ylabel('Concentrazione  %')

## Moneta 2

In [ ]:
ind = np.where(counts_2 > 0)[0]
x = energies_2[ind]
y = counts_2[ind]

result_5 = model_3.fit(y, params_3, x=x, weights=1/np.sqrt(y))
print(result_5.fit_report(show_correl=False))

In [ ]:
params_5 = result_5.params

amplitudes_values_2 = [param.value / REBIN / width for param in params_5.values() if 'Ka_amplitude' in param.name]
amplitudes_errors_2 = [param.stderr / REBIN / width for param in params_5.values() if 'Ka_amplitude' in param.name]

amplitudes_2 = unp.uarray(amplitudes_values_2, amplitudes_errors_2)
wts_2 = amplitudes_2 / amplitudes_2.sum() * 100

fig, ax = plt.subplots(figsize=(5,3), nrows=1, ncols=1, dpi=150)
ax.grid(ls=":")
ax.set_axisbelow(True)

ax.errorbar(range(3), unp.nominal_values(wts_2), unp.std_devs(wts_2), fmt='o', ms=4, label='XRF')
# ax.errorbar(range(3), [25,75,0], fmt='o', ms=4, label='Nominal')
ax.errorbar(range(3), [5,75,20], fmt='o', ms=4, label='Nominal')

ax.legend()
ax.set_xlim(-0.5,2.5)
ax.set_ylim(0,100)
ax.set_xticks([0,1,2])
ax.set_xticklabels(['Ni','Cu','Zn'])
ax.set_ylabel('Concentrazione  %')